In [1]:
import torch
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from models.ch_mdl_dolg_efficientnet import ArcFaceLoss
from models.eff import get_model
train = pd.read_csv('../whale/train.csv')
test = pd.read_csv('../whale/sample_submission.csv')
classes = list(set(train.individual_id))
classes_species = list(set(train.species))
sorted(classes)
sorted(classes_species)
display(train.head())
display(test.head())
print(len(classes))
print(len(classes_species))

,image,species,individual_id
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9
1,000562241d384d.jpg,humpback_whale,1a71fbb72250
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392


,image,predictions
0,000110707af0ba.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
1,0006287ec424cb.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
2,000809ecb2ccad.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
3,00098d1376dab2.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...
4,000b8d89c738bd.jpg,37c7aba965a5 114207cab555 a6e325d8e924 19fbb96...


15587
30


In [ ]:
import sys
import importlib
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.special import softmax
from joblib import Parallel, delayed
import seaborn as sns
import scipy as sp
def cos_similarity_matrix(a, b, eps=1e-8):
    a_n, b_n = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]
    a_norm = a / torch.max(a_n, eps * torch.ones_like(a_n))
    b_norm = b / torch.max(b_n, eps * torch.ones_like(b_n))
    sim_mt = torch.mm(a_norm, b_norm.transpose(0, 1))
    return sim_mt
def get_topk_cossim(test_emb, tr_emb, batchsize = 64, k=10, device='cuda:0',verbose=True):
    tr_emb = torch.tensor(tr_emb, dtype = torch.float32, device=torch.device(device))
    test_emb = torch.tensor(test_emb, dtype = torch.float32, device=torch.device(device))
    vals = []
    inds = []
    for test_batch in tqdm(test_emb.split(batchsize),disable=1-verbose):
        sim_mat = cos_similarity_matrix(test_batch, tr_emb)
        vals_batch, inds_batch = torch.topk(sim_mat, k=k, dim=1)
        vals += [vals_batch.detach().cpu()]
        inds += [inds_batch.detach().cpu()]
    vals = torch.cat(vals)
    inds = torch.cat(inds)
    return vals, inds
def get_topk_cossim_sub(test_emb, tr_emb, vals_x=None, batchsize = 64, k=10, device='cuda:0',verbose=True):
    tr_emb = torch.tensor(tr_emb, dtype = torch.float32, device=torch.device(device))
    test_emb = torch.tensor(test_emb, dtype = torch.float32, device=torch.device(device))
    #vals_x = torch.tensor(vals_x, dtype = torch.float32, device=torch.device(device))
    vals = []
    inds = []
    for test_batch in tqdm(test_emb.split(batchsize),disable=1-verbose):
        sim_mat = cos_similarity_matrix(test_batch, tr_emb)
        # sim_mat = torch.clamp(sim_mat,0,1) #- vals_x.repeat(sim_mat.shape[0], 1)
        
        vals_batch, inds_batch = torch.topk(sim_mat, k=k, dim=1)
        vals += [vals_batch.detach().cpu()]
        inds += [inds_batch.detach().cpu()]
    vals = torch.cat(vals)
    inds = torch.cat(inds)
    return vals, inds
def map_per_image(label, predictions):
    indexes = np.unique(predictions, return_index=True)[1]
    predictions = [predictions[index] for index in sorted(indexes)]
    t = np.where(predictions[:5]==label)[0]
    if len(t)>0:
        return 1 / (t[0] + 1)
    else:
        return 0.0

In [2]:
train['classes'] = train.groupby('individual_id').ngroup()
train['classes_species'] = train.groupby('species').ngroup()
train.head()

,image,species,individual_id,classes,classes_species
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9,12348,18
1,000562241d384d.jpg,humpback_whale,1a71fbb72250,1636,14
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b,5842,9
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063,4551,2
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392,8721,14


In [3]:
import os

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

TRAIN_PATH = '../whale/crop/train_images'
TEST_PATH = '../whale/crop/test_images'

In [4]:
# ====================================================
# CFG
# ====================================================
class CFG:
    debug=False
    apex=False
    print_freq=100
    num_workers=8
    model_name='tf_efficientnet_b5_ns'
    size=448
    scheduler='CosineAnnealingWarmRestarts' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=15
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    #T_max=10 # CosineAnnealingLR
    T_0=10 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=128
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    target_size=6
    target_col='classes'
    target_col2 = 'classes_species'
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=True
    inference=False
model_name = CFG.model_name
if CFG.debug:
    CFG.epochs = 1
    train = train.sample(n=1000, random_state=CFG.seed).reset_index(drop=True)

In [5]:
import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from tqdm import tqdm
from functools import partial

import cv2
from PIL import Image
from sklearn.metrics import f1_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD,AdamW
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer
from sklearn.random_projection import GaussianRandomProjection
from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose
    )
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import timm

import warnings 
warnings.filterwarnings('ignore')

if CFG.apex:
    from apex import amp

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
folds = train.copy()
Fold = StratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(folds, folds[CFG.target_col])):
    folds.loc[val_index, 'fold'] = int(n)
folds['fold'] = folds['fold'].astype(int)
print(folds.groupby(['fold', CFG.target_col]).size())

fold  classes
0     4          1
      10         2
      13         1
      18         1
      19         1
                ..
4     15575      1
      15577      1
      15578      6
      15583      1
      15584      1
Length: 29343, dtype: int64


In [7]:
# ====================================================
# Dataset
# ====================================================
class TrainDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image'].values
        self.labels = df['classes'].values
        self.labels_s = df['classes_species'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TRAIN_PATH}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        label = torch.tensor(self.labels[idx]).long()
        label_species = torch.tensor(self.labels_s[idx]).long()
        return image, label,label_species
    

class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TEST_PATH}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

In [8]:
# ====================================================
# Transforms
# ====================================================
def get_transforms(*, data):
    
    if data == 'train':
        return Compose([
            Resize(CFG.size, CFG.size),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

    elif data == 'valid':
        return Compose([
            Resize(CFG.size, CFG.size),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

In [9]:
def valid_fn(valid_loader, model, device):
    model.eval()
    emb = []
    targets = []
    start = end = time.time()
    for step, (images, labels,labels2) in enumerate(tqdm(valid_loader)):
        # measure data loading time
        images = images.to(device)
        labels = labels.to(device).long()
        labels2 = labels2.to(device).long()
        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
            y_preds = model.extract_feature(images).cpu().numpy()
            emb.append(y_preds)
            targets.append(labels.cpu().numpy())
    emb = np.concatenate(emb)
    targets = np.concatenate(targets)
    
    return emb,targets

In [10]:
def infer_loop(folds, fold,run_cv = False):
    
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index

    train_folds = folds.loc[trn_idx].reset_index(drop=True)
    valid_folds = folds.loc[val_idx].reset_index(drop=True)

    train_dataset = TrainDataset(train_folds, 
                                 transform=get_transforms(data='train'))
    valid_dataset = TrainDataset(valid_folds, 
                                 transform=get_transforms(data='valid'))

    train_loader = DataLoader(train_dataset, 
                              batch_size=CFG.batch_size, 
                              shuffle=False, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    valid_loader = DataLoader(valid_dataset, 
                              batch_size=CFG.batch_size, 
                              shuffle=False, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    model = get_model(backbone_name = model_name)
    model.to(device)
    model.load_state_dict(torch.load(OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best.pth')['model'])
    emb_v,targets_v = valid_fn(valid_loader, model, device)
    emb_t,targets_t = valid_fn(train_loader, model, device)
    res = {}
    res['emb_v'] = emb_v
    res['targets_v'] = targets_v
    res['emb_t'] = emb_t
    res['targets_t'] = targets_t
    if not run_cv: 
        return res
    tr_embeddings = results[0]['emb_t']
    val_embeddings = results[0]['emb_v']
    targets = results[0]['targets_v']
    targets_train = results[0]['targets_t']
    EMB_SIZE = 512
    vals_blend = []
    labels_blend = []
    inds_blend = []
    for i in range(1):
        vals, inds = get_topk_cossim_sub(val_embeddings[:,i*EMB_SIZE:(i+1)*EMB_SIZE], tr_embeddings[:,i*EMB_SIZE:(i+1)*EMB_SIZE], k=500)
        vals = vals.data.cpu().numpy()
        inds = inds.data.cpu().numpy()
        labels = np.concatenate([targets_train[inds[:,i]].reshape(-1,1) for i in range(inds.shape[1])], axis=1)

        vals_blend.append(vals)
        labels_blend.append(labels)
        inds_blend.append(inds)
    vals = np.concatenate(vals_blend, axis=1)
    inds = np.concatenate(inds_blend, axis=1)
    labels = np.concatenate(labels_blend, axis=1)
    M = []
    for row in range(len(vals)):
        m = map_per_image(targets[row],labels[row])
        M.append(m)
    return np.array(M).mean()

In [11]:
results = []
for fold in range(1,2):
    results.append(infer_loop(folds, fold))

100%|██████████| 319/319 [03:59<00:00,  1.33it/s]


In [19]:
tr_embeddings = results[0]['emb_t']
val_embeddings = results[0]['emb_v']
targets = results[0]['targets_v']
targets_train = results[0]['targets_t']
f = QuantileTransformer(output_distribution="normal")
f.fit(np.concatenate([val_embeddings],axis=0))
tr_embeddings = f.transform(tr_embeddings)
val_embeddings = f.transform(val_embeddings)

In [20]:
EMB_SIZE = 512
vals_blend = []
labels_blend = []
inds_blend = []
for i in range(1):
    
    # vals_nl, inds_nl = get_topk_cossim(tr_embeddings[:,i*EMB_SIZE:(i+1)*EMB_SIZE], nonlandmark_embeddings[:,i*EMB_SIZE:(i+1)*EMB_SIZE], k=5)
    # vals_nl = vals_nl[:,:].mean(axis=1).detach().cpu().numpy()
    
    vals, inds = get_topk_cossim_sub(val_embeddings[:,i*EMB_SIZE:(i+1)*EMB_SIZE], tr_embeddings[:,i*EMB_SIZE:(i+1)*EMB_SIZE], k=500)
    vals = vals.data.cpu().numpy()
    inds = inds.data.cpu().numpy()
    labels = np.concatenate([targets_train[inds[:,i]].reshape(-1,1) for i in range(inds.shape[1])], axis=1)
    
    vals_blend.append(vals)
    labels_blend.append(labels)
    inds_blend.append(inds)

100%|██████████| 160/160 [00:00<00:00, 942.45it/s]


In [21]:
vals = np.concatenate(vals_blend, axis=1)
inds = np.concatenate(inds_blend, axis=1)
labels = np.concatenate(labels_blend, axis=1)

In [22]:
M = []
for row in range(len(vals)):
    m = map_per_image(targets[row],labels[row])
    M.append(m)
print(np.array(M).mean())

0.5159090166878939
